## Initial

### Install required packages

In [ ]:
# %pip install -q -r requirement.txt

### Import Secret

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Azure OpenAI API
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

# LLM
AZURE_OPENAI_LLM_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_LLM_DEPLOYMENT_NAME')
AZURE_OPENAI_LLM_MODEL_NAME = os.getenv('AZURE_OPENAI_LLM_MODEL_NAME')

# OpenWeatherMap
OPENWEATHERMAP_API_KEY = os.getenv('OPENWEATHERMAP_API_KEY')

## Initialize Azure OpenAI

In [ ]:
from langchain_openai import AzureChatOpenAI

#remove the langsmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ.pop("LANGCHAIN_API_KEY", None)

llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_LLM_DEPLOYMENT_NAME, 
    model=AZURE_OPENAI_LLM_MODEL_NAME,
    temperature=0.5,
)

## Query LLM

In [ ]:
result = llm.invoke("How many 'r' are in the word raspberry?")
print(result.content)

In [ ]:
result = llm.invoke("Where was the 2024 cricket world?")
print(result.content)

## Create Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Define Tools

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import tool
import requests

@tool
def count_letter_in_word(word: str, letter: str) -> int:
    """Returns the number of times a letter appears in a word.
    Args:
        word: The word to search.
        letter: The letter to count."""
    return word.count(letter)

@tool("weather")
def get_weather(city: str) -> str:
    """Returns the current weather for a city in json format.
    Args:
        city: The city to get the weather for."""
    
    # Make a request to the OpenWeatherMap API
    response = requests.get(
        f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHERMAP_API_KEY}"
    )
    return response.json()

@tool
def export_notes(notes: str, file_name: str):
    """Exports notes to a file.
    Args:
        notes: The notes to export.
        file_name: The name of the file to export to."""
    try:
        with open(f"./results/{file_name}", "w") as file:
            file.write(notes)
    except Exception as e:
        print(f"An error occurred while exporting notes: {e}")


### Binding tools to LLM

In [ ]:
tools = [
    count_letter_in_word,
    DuckDuckGoSearchResults(),
    get_weather,
    export_notes
]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

## Create the Agent

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run Query with Agent 

In [ ]:
query = "How many 'r' are in the word 'raspberry'?"

results = list(agent_executor.stream({"input": query}))

length = results.__len__()
print("results: " + results[length - 1]['output'])

In [ ]:
query = "Where was the 2024 cricket world tournament and which team won?"

results = list(agent_executor.stream({"input": query}))

length = results.__len__()
print("results: " + results[length - 1]['output'])

In [ ]:
query = """
Lars is on vacation in Tobago. What is the weather like there in celsius?
And can you search for some outdoors activities depending on the weather?
Export the results and weather to a file in point form.
"""

results = list(agent_executor.stream({"input": query}))

length = results.__len__()
print("results: " + results[length - 1]['output'])